In [4]:
import numpy as np
import pyaudio
import time
import librosa
import matplotlib.pyplot as plt
import librosa.display

from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from PIL import Image, ImageFont, ImageDraw
import numpy as np

In [11]:
class AudioHandler(object):
    def __init__(self):
        self.FORMAT = pyaudio.paFloat32
        self.CHANNELS = 1
        self.RATE = 44100
        self.CHUNK = 1024 * 2
        self.RECORD_SECONDS = 1
        self.INPUT_DEVICE_INDEX = 1 # change this parametr to good work
        self.p = None
        self.stream = None

    def start(self):
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(format=self.FORMAT,
                                  channels=self.CHANNELS,
                                  rate=self.RATE,
                                  input=True,
                                  output=False,
                                  #stream_callback=self.callback,
                                  frames_per_buffer=self.CHUNK,
                                  input_device_index = self.INPUT_DEVICE_INDEX)

    def stop(self):
        self.stream.close()
        self.p.terminate()

    def callback(self, in_data, frame_count, time_info, flag):
        numpy_array = np.frombuffer(in_data, dtype=np.float32)
        librosa.feature.mfcc(numpy_array)
        return None, pyaudio.paContinue

    def mainloop(self):
        while (self.stream.is_active()): # if using button you can set self.stream to 0 (self.stream = 0), otherwise you can use a stop condition
            model = load_model(r'D:\save_1.h5')
            
            frames = []  
            for i in range(0, int(self.RATE / self.CHUNK * self.RECORD_SECONDS)):
                data = self.stream.read(self.CHUNK)
                frames.append(data)   
            frame = b''.join(frames)
            callback(self, frame, len(frames), )
            print('ok')
                
            time.sleep(2.0)


audio = AudioHandler()
audio.start()     # open the the stream
audio.mainloop()  # main operations with librosa
audio.stop()

ok
ok
ok
ok
ok
ok
ok
ok
ok


KeyboardInterrupt: 

In [4]:
def create_spectrogram(y, sample_rate):
    
    plt.interactive(False)
    #clip, sample_rate = librosa.load(filename, sr=None)
        
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(spectrogram, ref=np.max))

    
    
    #filename = path
    #plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    
    
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    
    return spectrogram
    del filename,clip,sample_rate,fig,ax,S

In [5]:
import numpy as np
import librosa

def feature_extract(y, sr):
    """
    Define function that takes in a file an returns features in an array
    """
    
    #get wave representation
    #y, sr = librosa.load(file)
    #print(y)    
    #determine if instruemnt is harmonic or percussive by comparing means
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    if np.mean(y_harmonic)>np.mean(y_percussive):
        harmonic=1
    else:
        harmonic=0
        
    #Mel-frequency cepstral coefficients (MFCCs)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    #temporal averaging
    mfcc=np.mean(mfcc,axis=1)
    
    #get the mel-scaled spectrogram
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)  
    #temporally average spectrogram
    spectrogram = np.mean(spectrogram, axis = 1)
    
    #compute chroma energy
    chroma = librosa.feature.chroma_cens(y=y, sr=sr)
    #temporally average chroma
    chroma = np.mean(chroma, axis = 1)
    
    #compute spectral contrast
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast = np.mean(contrast, axis= 1)
    
    return [harmonic, mfcc, spectrogram, chroma, contrast]

In [7]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paFloat32
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 1
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")
# print(type(frames))
# print(type(frames[0]))
# print(frames[0])


all_frames = b''.join(frames)
print(type(all_frames))

# byte_sample = []
# for chunk in frames: 
#     byte_sample.append(chunk)

# print(type(byte_sample[0]))
# print(byte_sample[0])

numpy_array = np.frombuffer(all_frames, dtype=np.float32)
#print(librosa.feature.mfcc(numpy_array))
#print(numpy_array)
feature = feature_extract(numpy_array, RATE)
#print(feature)
sp = create_spectrogram(numpy_array, RATE)


# import matplotlib.pyplot as plt
# fig, ax = plt.subplots()
# img = librosa.display.specshow(feature[0], x_axis='time', ax=ax)
# fig.colorbar(img, ax=ax)
# ax.set(title='MFCC')

stream.stop_stream()
stream.close()
p.terminate()


wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


* recording
* done recording
<class 'bytes'>


In [10]:
print(sp)

[[3.97069311e-09 2.41422149e-09 3.83781753e-06 ... 1.23305679e+02
  5.33592186e+01 1.51112785e+01]
 [1.38279799e-09 3.14439719e-09 6.37267203e-06 ... 2.40405457e+02
  1.09371216e+02 4.67931137e+01]
 [8.40854997e-10 6.09344686e-10 8.29826149e-06 ... 6.96133270e+01
  1.72022438e+01 9.78102016e+00]
 ...
 [5.88393112e-09 5.01833108e-09 3.38652590e-08 ... 1.48421077e-05
  1.67465914e-05 2.08872098e-05]
 [5.80504711e-09 7.03017955e-09 3.55521550e-08 ... 1.60732889e-04
  7.36468064e-05 6.42417799e-05]
 [8.71521433e-09 9.29739485e-09 1.81104589e-08 ... 5.98095474e-04
  2.01462477e-04 1.32240239e-04]]


In [11]:
librosa.display.specshow(librosa.power_to_db(sp, ref=np.max))

In [13]:
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
#from data_prep import valik
import pandas as pd
from sklearn.metrics import confusion_matrix
from PIL import Image, ImageFont, ImageDraw
import numpy as np

path = load_model(r'D:\save_1.h5')

model = load_model('/home/aigaf/Downloads/Telegram Desktop/save_1.h5')


# path_to_img = 'cut1.jpg'
# img = Image.open(path_to_img)
# img_arr = np.asarray(img)
img_arr = sp
# print(img_arr.shape, img_arr)
img_norm = img_arr/255
img_size = img_norm.resize((150, 150, 3))
img_compl = img_norm.reshape(-1, 150, 150, 3)
# print(img_compl)

#results = model.evaluate(valik)
#print('loss, accuracy =', results)


final_res = model.predict(img_compl)
print(final_res)
# data = {'y_Actual':    [ tuple(i) for i in valik[1][1]],
#         'y_Predicted': [ tuple(i) for i in final_res]
#         }
#
# df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
# print (df)

[[0.00170791 0.9983909 ]]
